# 非自洽 HF-GGA 一阶梯度

在这一份笔记中，我们会回顾 HF-GGA 的一阶梯度的推导与实现过程．

In [ ]:
%load_ext autoreload
%autoreload 2

import numpy as np
from pyscf import scf, gto, lib, grad, hessian, dft
import pyscf.hessian.rks
import pyscf.grad.rks
from functools import partial

import sys
sys.path.append('../../src')
from utilities import val_from_fchk
from hf_helper import HFHelper
from gga_helper import GGAHelper
from numeric_helper import NumericDiff

np.einsum = partial(np.einsum, optimize=["greedy", 1024 ** 3 * 2 / 8])
np.set_printoptions(5, linewidth=120, suppress=True)

In [ ]:
# !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!! NOTE !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
# For clear and simplicity, following code cell will be hidden and assumed to be executed!

mol = gto.Mole()
mol.atom = """
O  0.0  0.0  0.0
O  0.0  0.0  1.5
H  1.5  0.0  0.0
H  0.0  0.7  1.5
"""
mol.basis = "6-31G"
mol.verbose = 0
mol.build()

grids = dft.gen_grid.Grids(mol)
grids.atom_grid = (99, 590)
grids.becke_scheme = dft.gen_grid.stratmann
grids.prune = None
grids.build()

nmo = nao = mol.nao
natm = mol.natm
nocc = mol.nelec[0]
nvir = nmo - nocc
so = slice(0, nocc)
sv = slice(nocc, nmo)
sa = slice(0, nmo)

`hfh` 使用 HF 帮手，它将包含与 HF 有关的重要中间量，包括二阶 U 矩阵等．

In [ ]:
hfh = HFHelper(mol)
hfh.get_all()
mol_slice=hfh.mol_slice

`nch` 使用 GGA 帮手，但它不计算 SCF 过程；但我们将会需要其中的格点信息，以及其 $F_{\mu \nu}^\mathrm{n}$ 的信息．所有上标了 $\mathrm{n}$ 的项都是从非自洽泛函得来；而若不标记，则默认是从 HF 导出．

In [ ]:
nch = GGAHelper(mol, "b3lypg", grids, init_scf=False)

In [ ]:
nch.D = hfh.D
nch.C = hfh.C

`F_0_ao` $F_{\mu \nu}^\mathrm{n} = h_{\mu \nu} + (\mu \nu | \kappa \lambda) D_{\kappa \lambda} - \frac{1}{2} c_\mathrm{x}^\mathrm{n} (\mu \kappa | \nu \lambda) D_{\kappa \lambda} + v_{\mu \nu}^\mathrm{xc, n}[\rho]$

In [ ]:
nch.get_kerh()
nch.F_0_ao = nch.scf_eng.get_fock(dm=nch.D)
nch.F_0_mo = nch.C.T @ nch.F_0_ao @ nch.C
print()

## 一阶梯度：数值解

数值解的方式需要首先知道 HF-GGA 的解析能量．这与 [XYG3 能量](basic_gga.ipynb#XYG3-能量计算) 的计算过程相同．结果储存在 `nceng_diff` 中．

In [ ]:
def mol_to_nceng(mol):
    hfh = HFHelper(mol)
    nch = GGAHelper(mol, "b3lypg", grids, init_scf=False)
    return nch.scf_eng.energy_tot(dm=hfh.D)

In [ ]:
nceng_diff = NumericDiff(mol, mol_to_nceng).get_numdif()

In [ ]:
nceng_diff

## 一阶梯度：解析解

### U 矩阵直接求解

我们先简单回顾一下二阶梯度的导出过程．这里直接使用 $E_\mathrm{elec}$ 来表示电子态能量，即使它是 HF-GGA 过程的能量，但也不上标 $\mathrm{n}$．

$$
E_\mathrm{elec} = h_{\mu \nu} D_{\mu \nu} + \frac{1}{2} (\mu \nu | \kappa \lambda) D_{\mu \nu} D_{\kappa \lambda} - \frac{c_\mathrm{x}^\mathrm{n}}{4} (\mu \kappa | \nu \lambda) D_{\mu \nu} D_{\kappa \lambda} + f^\mathrm{n} \rho
$$

$$
\frac{\partial}{\partial A_t} E_\mathrm{elec} = \partial_{A_t} E_\mathrm{elec} + \partial_{A_t}^\mathrm{U} E_\mathrm{elec}
$$

`E_S` 一阶能量梯度 (Skeleton)

$$
\partial_{A_t} E_\mathrm{elec} = h_{\mu \nu}^{A_t} D_{\mu \nu} + \frac{1}{2} (\mu \nu | \kappa \lambda)^{A_t} D_{\mu \nu} D_{\kappa \lambda} - \frac{c_\mathrm{x}^\mathrm{n}}{4} (\mu \kappa | \nu \lambda)^{A_t} D_{\mu \nu} D_{\kappa \lambda} + \partial_{A_t} (f^\mathrm{n} \rho)
$$

$$
\partial_{A_t} (f^\mathrm{n} \rho) = f_\rho^\mathrm{n} \rho^{A_t} + 2 f_\gamma^\mathrm{n} \rho_r \rho_r^{A_t}
$$

In [ ]:
E_S = (
    +                 np.einsum("Atuv, uv -> At", hfh.H_1_ao, hfh.D)
    + 0.5 *           np.einsum("Atuvkl, uv, kl -> At", hfh.eri1_ao, hfh.D, hfh.D)
    - 0.25 * nch.cx * np.einsum("Atukvl, uv, kl -> At", hfh.eri1_ao, hfh.D, hfh.D)
    +                 np.einsum("g, Atg -> At", nch.kerh.fr, nch.grdh.A_rho_1)
    + 2 *             np.einsum("g, rg, Atrg -> At", nch.kerh.fg, nch.grdh.rho_1, nch.grdh.A_rho_2)
)

`E_U_byU` 一阶能量梯度 (U)，通过 U 矩阵获得

$$
\partial_{A_t}^\mathrm{U} E_\mathrm{elec} = 4 U_{pi}^{A_t} F_{pi}^\mathrm{n}
$$

In [ ]:
E_U_byU = 4 * np.einsum("Atpi, pi -> At", hfh.U_1[:, :, :, so], nch.F_0_mo[:, so])

`E_1_byU` 总一阶能量梯度，通过 U 矩阵获得

In [ ]:
E_1_byU = E_S + E_U_byU + nch.scf_grad.grad_nuc()
np.allclose(E_1_byU, nceng_diff, atol=1e-7)

我们刚才在推导 $\partial_{A_t}^\mathrm{U} E_\mathrm{elec}$ 过程中跳步了．我们现在对其说明如下．这里利用到重叠与交换积分计算中，$\mu, \nu$ 与 $\kappa, \lambda$ 的可交换性．

\begin{align}
\partial_{A_t}^\mathrm{U} E_\mathrm{elec} &= \partial_{A_t}^\mathrm{U} D_{\mu \nu} \left( h_{\mu \nu} + (\mu \nu | \kappa \lambda) D_{\kappa \lambda} - \frac{c_\mathrm{x}^\mathrm{n}}{2} (\mu \kappa | \nu \lambda) D_{\kappa \lambda} \right) + \partial_{A_t}^\mathrm{U} (f^\mathrm{n} \rho) \\
&= 4 U_{pi}^{A_t} C_{\mu p} C_{\nu i} \left( h_{\mu \nu} + (\mu \nu | \kappa \lambda) D_{\kappa \lambda} - \frac{c_\mathrm{x}^\mathrm{n}}{2} (\mu \kappa | \nu \lambda) D_{\kappa \lambda} \right) \\
&\quad \mathrel+ 4 U_{pi}^{A_t} C_{\mu p} C_{\nu i} \left( f_\rho^\mathrm{n} \phi_\mu \phi_\nu + 2 f_\gamma^\mathrm{n} \rho_r ( \phi_{r \mu} \phi_\nu + \phi_\mu \phi_{r \nu} ) \right) \\
&= 4 U_{pi}^{A_t} C_{\mu p} C_{\nu i} F_{\mu \nu}^\mathrm{n} = 4 U_{pi}^{A_t} F_{pi}^\mathrm{n}
\end{align}

这个关系式非常重要，它的存在大大简化了公式的复杂性．我们以后会发现，二阶 HF-GGA 梯度公式最复杂的部分并不是 U-U 或者 Skeleton-U 导数，而是 Skeleton-Skeleton 导数 (即使这在 PySCF 中也能用高级函数来获得)；因为 Skeleton-Skeleton 导数无法用已有的记号进行简化．

### Z-Vector 方程求解

上述过程需要求解多次一阶 CP-HF 方程．我们说，一阶 CP-HF 方程的复杂度是 $T O(N^4)$；其中的四次方是 ERI 与 U 矩阵作张量缩并时所必要的计算量，而 $T$ 代表迭代次数．但是，如果我们要解 $U$ 矩阵，我们不应当认为总计算量是 $T O(N^4)$，而应当认为是 $T O(N^5)$，这是因为 U 矩阵一共有原子数乘以 3 个；我们需要解原子数乘以 3 个 CP-HF 方程；而基组数与原子数可以看成是一起增长的，因此原子数若作为一个维度可以当做基组．

下述的 Z-Vector 方程过程事实上也是 $O(N^5)$ 计算量；但我们指出，它没有迭代次数的前置系数．Z-Vector 过程通过一次 CP-HF ($T O(N^4)$) 过程求解了 $Z_{ai}$ (其意义我们将马上叙述)，随后与 $O(N^5)$ 计算复杂度的 $B_{ai}^{A_t}$ 作张量缩并．这是更为可取的梯度求解思路．

现在我们来简单讲述一下 Z-Vector 方程的导出过程．首先，我们需要将梯度化为下式 (利用 $S_{ij}^{A_t} + U_{ij}^{A_t} + U_{ji}^{A_t} = 0$)

$$
\partial_{A_t}^\mathrm{U} E_\mathrm{elec} = 4 U_{ai}^{A_t} F_{ai}^\mathrm{n} - 2 S_{ki}^{A_t} F_{ki}^\mathrm{n}
$$

我们参考 [以前文档](u_rhf.ipynb#矩阵求逆) 中的记号，定义 $A_{ai, bj}' = - A_{ai, bj} - \delta_{ab} \delta_{ij} (\varepsilon_b - \varepsilon_j))$．那么，CP-HF 方程可以写为

$$
A_{ai, bj}' U_{bj}^{A_t} = B_{ai}^{A_t}
$$

并且，

$$
U_{ai}^{A_t} = (\mathbf{A}')^{-1}_{ai, bj} B_{bj}^{A_t}
$$

注意到我们想求 $U_{ai}^{A_t} F_{ai}^\mathrm{n}$，那么我们对等式两边分别乘以 $F_{ai}^\mathrm{n}$ 并对 $a, i$ 求和：

$$
U_{ai}^{A_t} F_{ai}^\mathrm{n} = F_{ai}^\mathrm{n} (\mathbf{A}')^{-1}_{ai, bj} B_{bj}^{A_t}
$$

现在我们定义

$$
Z_{ai} = F_{ai}^\mathrm{n} (\mathbf{A}')^{-1}_{ai, bj}
$$

即

$$
A_{ai, bj} Z_{bj} = F_{ai}^\mathrm{n}
$$

那么

$$
U_{ai}^{A_t} F_{ai}^\mathrm{n} = Z_{bj} B_{bj}^{A_t}
$$

In [ ]:
Z = scf.cphf.solve(hfh.Ax0_Core(sv, so, sv, so), hfh.e, hfh.mo_occ, nch.F_0_mo[sv, so])[0]

In [ ]:
E_U = (
    + 4 * np.einsum("Atai, ai -> At", hfh.B_1[:, :, sv, so], Z)
    - 2 * np.einsum("Atki, ki -> At", hfh.S_1_mo[:, :, so, so], nch.F_0_mo[so, so])
)

In [ ]:
E_1 = E_S + E_U + nch.scf_grad.grad_nuc()
np.allclose(E_1, nceng_diff, atol=1e-7)

### 外部程序

现在已经有外部程序，可以直接计算非自洽 HF-GGA 的一阶梯度．

In [ ]:
from ncgga_engine import NCGGAEngine

In [ ]:
ncengine = NCGGAEngine(hfh, nch)
np.allclose(ncengine.get_E_1(), E_1)